# Web Sraping Bayut using Selenium

## Imports needed

In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np

In [37]:
webdriver = Chrome()

## Extracting Links

### Day 1

In [38]:
links = []
webdriver.get("https://www.bayut.eg/en/alexandria/properties-for-sale")
twait = WebDriverWait(webdriver, 10)
objs = twait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_8969fafd")))
for obj in objs:
    links.append(obj.get_attribute("href"))
for i in range(2, 64):
    webdriver.get(f"https://www.bayut.eg/en/alexandria/properties-for-sale/page-{i}/")
    twait = WebDriverWait(webdriver, 10)
    objs = twait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_8969fafd")))
    for obj in objs:
        links.append(obj.get_attribute("href"))
print(len(links))

3150


In [39]:
len(links), links.count(None)

(3150, 0)

In [41]:
links[1]

'https://www.bayut.eg/en/property/details-502678081.html'

In [43]:
AllTogether = []
for link in links:
    twait = WebDriverWait(webdriver, 10)
    webdriver.get(link)
    location = twait.until(EC.presence_of_element_located((By.CLASS_NAME, "_8fd912b5"))).text
    price = twait.until(EC.presence_of_element_located((By.CLASS_NAME, "b59a4b92"))).text
    MoreDetails = twait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_3458a9d4")))
    details = []
    for detail in MoreDetails:
        details.append(detail.text)
    AllDetails = details.extend([location, price])
    AllTogether.append(details)

In [49]:
type(AllTogether), len(AllTogether)

(list, 3150)

In [57]:
CurrentVersion = pd.DataFrame(AllTogether, columns=['Number of Beds', 'Number of Paths', 'Area of Apartment', 'Location', 'Price'])
CurrentVersion['Links'] = links
CurrentVersion

,Number of Beds,Number of Paths,Area of Apartment,Location,Price,Links
0,4 Beds,2 Baths,400 Sq. M.,"Sporting, Alexandria","12,000,000",https://www.bayut.eg/en/property/details-50262...
1,4 Beds,3 Baths,217 Sq. M.,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...
2,4 Beds,3 Baths,217 Sq. M.,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...
3,4 Beds,3 Baths,217 Sq. M.,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...
4,2 Beds,2 Baths,119 Sq. M.,"Sawari, Moharam Bik, Alexandria","5,497,000",https://www.bayut.eg/en/property/details-50266...
...,...,...,...,...,...,...
3145,2 Beds,1 Bath,100 Sq. M.,"Mandara, Alexandria","1,950,000",https://www.bayut.eg/en/property/details-50264...
3146,3 Beds,2 Baths,150 Sq. M.,"Laurent, Alexandria","3,950,000",https://www.bayut.eg/en/property/details-50259...
3147,3 Beds,2 Baths,150 Sq. M.,"Laurent, Alexandria","3,950,000",https://www.bayut.eg/en/property/details-50259...
3148,5 Beds,5 Baths,400 Sq. M.,"King Mariout, Alexandria","10,000,000",https://www.bayut.eg/en/property/details-50258...


In [58]:
CurrentVersion.to_csv("BayutData(NotCompleted).csv", index=False)

### Day 2

In [ ]:
PreviousData = pd.read_csv("BayutData(NotCompleted).csv")
PreviousData['Links']

0       https://www.bayut.eg/en/property/details-50262...
1       https://www.bayut.eg/en/property/details-50267...
2       https://www.bayut.eg/en/property/details-50267...
3       https://www.bayut.eg/en/property/details-50267...
4       https://www.bayut.eg/en/property/details-50266...
                              ...                        
3145    https://www.bayut.eg/en/property/details-50264...
3146    https://www.bayut.eg/en/property/details-50259...
3147    https://www.bayut.eg/en/property/details-50259...
3148    https://www.bayut.eg/en/property/details-50258...
3149    https://www.bayut.eg/en/property/details-50258...
Name: Links, Length: 3150, dtype: object

In [24]:
webdriver = Chrome()

In [9]:
Links = list(np.array(PreviuosData['Links']))

In [ ]:
SellerData = []
ErrorLinks = []
counter = 1
for link in Links:
    twait = WebDriverWait(webdriver, 10)
    webdriver.get(link)
    try:
        SellerLink = twait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Agent name"]'))).get_attribute("href")
        if SellerLink:
            webdriver.get(SellerLink)
            SellerName = twait.until(EC.presence_of_element_located((By.CLASS_NAME, "b5373187"))).text.title()
            SellerData.append([SellerName, SellerLink])
        else:
            SellerName = twait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Agent name"]'))).text.title()
            SellerData.append([SellerName, np.nan])
    except Exception:
        ErrorLinks.append(link)
        SellerData.append([np.nan, np.nan])
        counter += 1
#    except Exception:
#        SellerName = twait.until(EC.presence_of_element_located((By.CLASS_NAME, "c335648a"))).text.title()
#        SellerData.extend([SellerName, np.nan])
#    finally:
#        counter += 1
#        if counter % 1000 == 0:
#            print(f"{counter} Links has been scraped")
#            print("=" * 50)
#        elif counter == len(Links):
#            print("Scraping Completed")
    counter += 1
    if counter % 1000 == 0:
        print(f"{counter} Links has been scraped")
        print("=" * 50)
    elif counter == len(Links):
        print("Scraping Completed")

1000 Links has been scraped


ReadTimeoutError: HTTPConnectionPool(host='localhost', port=49919): Read timed out. (read timeout=120)

In [49]:
for i in range(len(Links)):
    if Links[i] in ErrorLinks or i < len(SellerData):
        continue
    twait = WebDriverWait(webdriver, 10)
    webdriver.get(Links[i])
    try:
        SellerLink = twait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Agent name"]'))).get_attribute("href")
        if SellerLink:
            webdriver.get(SellerLink)
            SellerName = twait.until(EC.presence_of_element_located((By.CLASS_NAME, "b5373187"))).text.title()
            SellerData.append([SellerName, SellerLink])
        else:
            SellerName = twait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Agent name"]'))).text.title()
            SellerData.append([SellerName, np.nan])
    except Exception:
        ErrorLinks.append(link)
        SellerData.append([np.nan, np.nan])
        counter += 1
#    except Exception:
#        SellerName = twait.until(EC.presence_of_element_located((By.CLASS_NAME, "c335648a"))).text.title()
#        SellerData.extend([SellerName, np.nan])
#    finally:
#        counter += 1
#        if counter % 1000 == 0:
#            print(f"{counter} Links has been scraped")
#            print("=" * 50)
#        elif counter == len(Links):
#            print("Scraping Completed")
    counter += 1
    if counter % 1000 == 0:
        print(f"{counter} Links has been scraped")
        print("=" * 50)
    elif counter == len(Links):
        print("Scraping Completed")

Skipping 1 link
Skipping 2 link
Skipping 3 link
Skipping 4 link
Skipping 5 link
Skipping 6 link
Skipping 7 link
Skipping 8 link
Skipping 9 link
Skipping 10 link
Skipping 11 link
Skipping 12 link
Skipping 13 link
Skipping 14 link
Skipping 15 link
Skipping 16 link
Skipping 17 link
Skipping 18 link
Skipping 19 link
Skipping 20 link
Skipping 21 link
Skipping 22 link
Skipping 23 link
Skipping 24 link
Skipping 25 link
Skipping 26 link
Skipping 27 link
Skipping 28 link
Skipping 29 link
Skipping 30 link
Skipping 31 link
Skipping 32 link
Skipping 33 link
Skipping 34 link
Skipping 35 link
Skipping 36 link
Skipping 37 link
Skipping 38 link
Skipping 39 link
Skipping 40 link
Skipping 41 link
Skipping 42 link
Skipping 43 link
Skipping 44 link
Skipping 45 link
Skipping 46 link
Skipping 47 link
Skipping 48 link
Skipping 49 link
Skipping 50 link
Skipping 51 link
Skipping 52 link
Skipping 53 link
Skipping 54 link
Skipping 55 link
Skipping 56 link
Skipping 57 link
Skipping 58 link
Skipping 59 link
Skippi

In [51]:
len(SellerData)

3150

In [50]:
ErrorLinks

['https://www.bayut.eg/en/property/details-502684335.html',
 'https://www.bayut.eg/en/property/details-502684335.html']

In [ ]:
PreviousData['Seller Name'] = np.array([item[0] if item[0] else "Not Found" for item in SellerData])
PreviousData['Seller Link'] = np.array([item[1] if item[1] else "Not Found" for item in SellerData])
PreviousData.to_csv("BayutData(Before Editing).csv", index=False)

## Day 3

In [3]:
LastDayEd = pd.read_csv("BayutData(Before Editing).csv")
LastDayEd

,Number of Beds,Number of Paths,Area of Apartment,Location,Price,Links,Seller Name,Seller Link
0,4 Beds,2 Baths,400 Sq. M.,"Sporting, Alexandria","12,000,000",https://www.bayut.eg/en/property/details-50262...,Mohamed Abdelsalam,https://www.bayut.eg/en/brokers/mohamed-abdels...
1,4 Beds,3 Baths,217 Sq. M.,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...
2,4 Beds,3 Baths,217 Sq. M.,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...
3,4 Beds,3 Baths,217 Sq. M.,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...
4,2 Beds,2 Baths,119 Sq. M.,"Sawari, Moharam Bik, Alexandria","5,497,000",https://www.bayut.eg/en/property/details-50266...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...
...,...,...,...,...,...,...,...,...
3145,2 Beds,1 Bath,100 Sq. M.,"Mandara, Alexandria","1,950,000",https://www.bayut.eg/en/property/details-50264...,محمد عادل,NaN
3146,3 Beds,2 Baths,150 Sq. M.,"Laurent, Alexandria","3,950,000",https://www.bayut.eg/en/property/details-50259...,Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...
3147,3 Beds,2 Baths,150 Sq. M.,"Laurent, Alexandria","3,950,000",https://www.bayut.eg/en/property/details-50259...,Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...
3148,5 Beds,5 Baths,400 Sq. M.,"King Mariout, Alexandria","10,000,000",https://www.bayut.eg/en/property/details-50258...,عاشور,NaN


In [4]:
CorruptedData = LastDayEd.loc[LastDayEd['Location'].isna()]

In [5]:
pd.options.mode.chained_assignment = None  # default='warn'
CorruptedData['Price'] = CorruptedData['Area of Apartment']
CorruptedData['Location'] = CorruptedData['Number of Paths']
CorruptedData['Area of Apartment'] = CorruptedData['Number of Beds']
CorruptedData['Number of Beds'] = np.nan
CorruptedData['Number of Paths'] = np.nan
CorruptedData

,Number of Beds,Number of Paths,Area of Apartment,Location,Price,Links,Seller Name,Seller Link
24,NaN,NaN,"1,000 Sq. M.","Moharam Bik, Alexandria","3,000,000",https://www.bayut.eg/en/property/details-50246...,شركة الفتح المبين للتسويق العقاري,NaN
25,NaN,NaN,"1,000 Sq. M.","Moharam Bik, Alexandria","3,000,000",https://www.bayut.eg/en/property/details-50246...,شركة الفتح المبين للتسويق العقاري,NaN
162,NaN,NaN,320 Sq. M.,"Moharam Bik, Alexandria","1,850,000",https://www.bayut.eg/en/property/details-20220...,Mohamed Khaled,NaN
163,NaN,NaN,320 Sq. M.,"Moharam Bik, Alexandria","1,850,000",https://www.bayut.eg/en/property/details-20220...,Mohamed Khaled,NaN
176,NaN,NaN,276 Sq. M.,"Borg al-Arab, Alexandria","1,750,000",https://www.bayut.eg/en/property/details-50232...,Quality For Real Estate,NaN
177,NaN,NaN,276 Sq. M.,"Borg al-Arab, Alexandria","1,750,000",https://www.bayut.eg/en/property/details-50232...,Quality For Real Estate,NaN
638,NaN,NaN,200 Sq. M.,"Moharam Bik, Alexandria","500,000",https://www.bayut.eg/en/property/details-20106...,Mohamed Khaled,NaN
639,NaN,NaN,200 Sq. M.,"Moharam Bik, Alexandria","500,000",https://www.bayut.eg/en/property/details-20106...,Mohamed Khaled,NaN
1244,NaN,NaN,900 Sq. M.,"Maamoura, Alexandria","35,000",https://www.bayut.eg/en/property/details-50268...,Waleed Silver 25,NaN
1245,NaN,NaN,900 Sq. M.,"Maamoura, Alexandria","35,000",https://www.bayut.eg/en/property/details-50268...,Waleed Silver 25,NaN


In [6]:
pd.set_option('chained_assignment', None)
LastDayEd['Number of Beds'].loc[LastDayEd['Location'].isna()] = CorruptedData['Number of Beds']
LastDayEd['Number of Paths'].loc[LastDayEd['Location'].isna()] = CorruptedData['Number of Paths']
LastDayEd['Area of Apartment'].loc[LastDayEd['Location'].isna()] = CorruptedData['Area of Apartment']
LastDayEd['Price'].loc[LastDayEd['Location'].isna()] = CorruptedData['Price']
LastDayEd['Location'].loc[LastDayEd['Location'].isna()] = CorruptedData['Location']
LastDayEd[LastDayEd['Number of Beds'].isna()]

C:\Users\mazen\AppData\Local\Temp\ipykernel_32716\4236891676.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  LastDayEd['Number of Beds'].loc[LastDayEd['Location'].isna()] = CorruptedData['Number of Beds']
C:\Users\mazen\AppData\Local\Tem

,Number of Beds,Number of Paths,Area of Apartment,Location,Price,Links,Seller Name,Seller Link
24,NaN,NaN,"1,000 Sq. M.","Moharam Bik, Alexandria","3,000,000",https://www.bayut.eg/en/property/details-50246...,شركة الفتح المبين للتسويق العقاري,NaN
25,NaN,NaN,"1,000 Sq. M.","Moharam Bik, Alexandria","3,000,000",https://www.bayut.eg/en/property/details-50246...,شركة الفتح المبين للتسويق العقاري,NaN
162,NaN,NaN,320 Sq. M.,"Moharam Bik, Alexandria","1,850,000",https://www.bayut.eg/en/property/details-20220...,Mohamed Khaled,NaN
163,NaN,NaN,320 Sq. M.,"Moharam Bik, Alexandria","1,850,000",https://www.bayut.eg/en/property/details-20220...,Mohamed Khaled,NaN
176,NaN,NaN,276 Sq. M.,"Borg al-Arab, Alexandria","1,750,000",https://www.bayut.eg/en/property/details-50232...,Quality For Real Estate,NaN
177,NaN,NaN,276 Sq. M.,"Borg al-Arab, Alexandria","1,750,000",https://www.bayut.eg/en/property/details-50232...,Quality For Real Estate,NaN
638,NaN,NaN,200 Sq. M.,"Moharam Bik, Alexandria","500,000",https://www.bayut.eg/en/property/details-20106...,Mohamed Khaled,NaN
639,NaN,NaN,200 Sq. M.,"Moharam Bik, Alexandria","500,000",https://www.bayut.eg/en/property/details-20106...,Mohamed Khaled,NaN
1244,NaN,NaN,900 Sq. M.,"Maamoura, Alexandria","35,000",https://www.bayut.eg/en/property/details-50268...,Waleed Silver 25,NaN
1245,NaN,NaN,900 Sq. M.,"Maamoura, Alexandria","35,000",https://www.bayut.eg/en/property/details-50268...,Waleed Silver 25,NaN


In [7]:
LastDayEd[LastDayEd['Location'].isna()] # Problem Solved

,Number of Beds,Number of Paths,Area of Apartment,Location,Price,Links,Seller Name,Seller Link


In [17]:
LastDayEd[['Seller Name', 'Seller Link']].loc[LastDayEd['Seller Name'].isna()] = "Not Found"


C:\Users\mazen\AppData\Local\Temp\ipykernel_32716\1653477656.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  LastDayEd[['Seller Name', 'Seller Link']].loc[LastDayEd['Seller Name'].isna()] = "Not Found"


In [44]:
mask = LastDayEd['Seller Name'].isna() & LastDayEd['Seller Link'].isna()
LastDayEd.loc[mask, ['Seller Name', 'Seller Link']] = "Not Found"

In [45]:
LastDayEd[['Seller Link', 'Seller Name']].loc[LastDayEd['Seller Name'].isna()]

,Seller Link,Seller Name


In [48]:
LastDayEd.loc[LastDayEd['Seller Link'].isna(), ['Seller Link']] = "No Link Was Provided"

In [49]:
LastDayEd

,Number of Beds,Number of Paths,Area of Apartment,Location,Price,Links,Seller Name,Seller Link
0,4 Beds,2 Baths,400 Sq. M.,"Sporting, Alexandria","12,000,000",https://www.bayut.eg/en/property/details-50262...,Mohamed Abdelsalam,https://www.bayut.eg/en/brokers/mohamed-abdels...
1,4 Beds,3 Baths,217 Sq. M.,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...
2,4 Beds,3 Baths,217 Sq. M.,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...
3,4 Beds,3 Baths,217 Sq. M.,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...
4,2 Beds,2 Baths,119 Sq. M.,"Sawari, Moharam Bik, Alexandria","5,497,000",https://www.bayut.eg/en/property/details-50266...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...
...,...,...,...,...,...,...,...,...
3145,2 Beds,1 Bath,100 Sq. M.,"Mandara, Alexandria","1,950,000",https://www.bayut.eg/en/property/details-50264...,محمد عادل,No Link Was Provided
3146,3 Beds,2 Baths,150 Sq. M.,"Laurent, Alexandria","3,950,000",https://www.bayut.eg/en/property/details-50259...,Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...
3147,3 Beds,2 Baths,150 Sq. M.,"Laurent, Alexandria","3,950,000",https://www.bayut.eg/en/property/details-50259...,Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...
3148,5 Beds,5 Baths,400 Sq. M.,"King Mariout, Alexandria","10,000,000",https://www.bayut.eg/en/property/details-50258...,عاشور,No Link Was Provided


In [ ]:
import re

# Convert 'Number of Beds' to numeric (extract digits)
LastDayEd['Number of Beds'] = LastDayEd['Number of Beds'].str.extract(r'(\d+)').astype(float)

# Convert 'Number of Paths' to numeric and rename to 'Number of Baths'
LastDayEd['Number of Baths'] = LastDayEd['Number of Paths'].str.extract(r'(\d+)').astype(float)
LastDayEd.drop(columns=['Number of Paths'], inplace=True)

# Convert 'Area of Apartment' to numeric (extract digits, handle commas)
LastDayEd['Area of Apartment'] = LastDayEd['Area of Apartment'].str.replace(',', '').str.extract(r'(\d+\.?\d*)').astype(float)

# Convert 'Price' to numeric (extract digits, handle commas)
LastDayEd['Price'] = LastDayEd['Price'].str.replace(',', '').str.extract(r'(\d+\.?\d*)').astype(float)

,Number of Beds,Area of Apartment,Location,Price,Links,Seller Name,Seller Link,Number of Baths
0,4.0,400.0,"Sporting, Alexandria","12,000,000",https://www.bayut.eg/en/property/details-50262...,Mohamed Abdelsalam,https://www.bayut.eg/en/brokers/mohamed-abdels...,2.0
1,4.0,217.0,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,3.0
2,4.0,217.0,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,3.0
3,4.0,217.0,"Alex West, Agami, Alexandria","8,250,000",https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,3.0
4,2.0,119.0,"Sawari, Moharam Bik, Alexandria","5,497,000",https://www.bayut.eg/en/property/details-50266...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,2.0


In [52]:
LastDayEd

,Number of Beds,Area of Apartment,Location,Price,Links,Seller Name,Seller Link,Number of Baths
0,4.0,400.0,"Sporting, Alexandria",12000000.0,https://www.bayut.eg/en/property/details-50262...,Mohamed Abdelsalam,https://www.bayut.eg/en/brokers/mohamed-abdels...,2.0
1,4.0,217.0,"Alex West, Agami, Alexandria",8250000.0,https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,3.0
2,4.0,217.0,"Alex West, Agami, Alexandria",8250000.0,https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,3.0
3,4.0,217.0,"Alex West, Agami, Alexandria",8250000.0,https://www.bayut.eg/en/property/details-50267...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,3.0
4,2.0,119.0,"Sawari, Moharam Bik, Alexandria",5497000.0,https://www.bayut.eg/en/property/details-50266...,Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,2.0
...,...,...,...,...,...,...,...,...
3145,2.0,100.0,"Mandara, Alexandria",1950000.0,https://www.bayut.eg/en/property/details-50264...,محمد عادل,No Link Was Provided,1.0
3146,3.0,150.0,"Laurent, Alexandria",3950000.0,https://www.bayut.eg/en/property/details-50259...,Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...,2.0
3147,3.0,150.0,"Laurent, Alexandria",3950000.0,https://www.bayut.eg/en/property/details-50259...,Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...,2.0
3148,5.0,400.0,"King Mariout, Alexandria",10000000.0,https://www.bayut.eg/en/property/details-50258...,عاشور,No Link Was Provided,5.0


In [53]:
LastDayEd = LastDayEd[['Number of Beds', 'Number of Baths', 'Area of Apartment', 'Price', 'Location', 'Seller Name', 'Seller Link', 'Links']]
LastDayEd

,Number of Beds,Number of Baths,Area of Apartment,Price,Location,Seller Name,Seller Link,Links
0,4.0,2.0,400.0,12000000.0,"Sporting, Alexandria",Mohamed Abdelsalam,https://www.bayut.eg/en/brokers/mohamed-abdels...,https://www.bayut.eg/en/property/details-50262...
1,4.0,3.0,217.0,8250000.0,"Alex West, Agami, Alexandria",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50267...
2,4.0,3.0,217.0,8250000.0,"Alex West, Agami, Alexandria",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50267...
3,4.0,3.0,217.0,8250000.0,"Alex West, Agami, Alexandria",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50267...
4,2.0,2.0,119.0,5497000.0,"Sawari, Moharam Bik, Alexandria",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50266...
...,...,...,...,...,...,...,...,...
3145,2.0,1.0,100.0,1950000.0,"Mandara, Alexandria",محمد عادل,No Link Was Provided,https://www.bayut.eg/en/property/details-50264...
3146,3.0,2.0,150.0,3950000.0,"Laurent, Alexandria",Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...,https://www.bayut.eg/en/property/details-50259...
3147,3.0,2.0,150.0,3950000.0,"Laurent, Alexandria",Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...,https://www.bayut.eg/en/property/details-50259...
3148,5.0,5.0,400.0,10000000.0,"King Mariout, Alexandria",عاشور,No Link Was Provided,https://www.bayut.eg/en/property/details-50258...


In [54]:
LastDayEd['Area of Apartment (Sq. m)'] = LastDayEd['Area of Apartment']
LastDayEd['Price (EGP)'] = LastDayEd['Price']
LastDayEd.drop(columns=['Area of Apartment', 'Price'])
LastDayEd = LastDayEd[['Number of Beds', 'Number of Baths', 'Area of Apartment (Sq. m)', 'Price (EGP)', 'Location', 'Seller Name', 'Seller Link', 'Links']]
LastDayEd

,Number of Beds,Number of Baths,Area of Apartment (Sq. m),Price (EGP),Location,Seller Name,Seller Link,Links
0,4.0,2.0,400.0,12000000.0,"Sporting, Alexandria",Mohamed Abdelsalam,https://www.bayut.eg/en/brokers/mohamed-abdels...,https://www.bayut.eg/en/property/details-50262...
1,4.0,3.0,217.0,8250000.0,"Alex West, Agami, Alexandria",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50267...
2,4.0,3.0,217.0,8250000.0,"Alex West, Agami, Alexandria",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50267...
3,4.0,3.0,217.0,8250000.0,"Alex West, Agami, Alexandria",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50267...
4,2.0,2.0,119.0,5497000.0,"Sawari, Moharam Bik, Alexandria",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50266...
...,...,...,...,...,...,...,...,...
3145,2.0,1.0,100.0,1950000.0,"Mandara, Alexandria",محمد عادل,No Link Was Provided,https://www.bayut.eg/en/property/details-50264...
3146,3.0,2.0,150.0,3950000.0,"Laurent, Alexandria",Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...,https://www.bayut.eg/en/property/details-50259...
3147,3.0,2.0,150.0,3950000.0,"Laurent, Alexandria",Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...,https://www.bayut.eg/en/property/details-50259...
3148,5.0,5.0,400.0,10000000.0,"King Mariout, Alexandria",عاشور,No Link Was Provided,https://www.bayut.eg/en/property/details-50258...


In [55]:
# Split 'Location' into 'Region' and 'City'
LastDayEd['Region'] = LastDayEd['Location'].str.replace(', Alexandria', '', regex=False)
LastDayEd['City'] = 'Alexandria'

# Move columns to desired order
LastDayEd = LastDayEd[['Number of Beds', 'Number of Baths', 'Area of Apartment (Sq. m)', 'Price (EGP)', 'City', 'Region', 'Seller Name', 'Seller Link', 'Links']]
LastDayEd

,Number of Beds,Number of Baths,Area of Apartment (Sq. m),Price (EGP),City,Region,Seller Name,Seller Link,Links
0,4.0,2.0,400.0,12000000.0,Alexandria,Sporting,Mohamed Abdelsalam,https://www.bayut.eg/en/brokers/mohamed-abdels...,https://www.bayut.eg/en/property/details-50262...
1,4.0,3.0,217.0,8250000.0,Alexandria,"Alex West, Agami",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50267...
2,4.0,3.0,217.0,8250000.0,Alexandria,"Alex West, Agami",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50267...
3,4.0,3.0,217.0,8250000.0,Alexandria,"Alex West, Agami",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50267...
4,2.0,2.0,119.0,5497000.0,Alexandria,"Sawari, Moharam Bik",Ahmed Elsayed,https://www.bayut.eg/en/brokers/ahmed-elsayed-...,https://www.bayut.eg/en/property/details-50266...
...,...,...,...,...,...,...,...,...,...
3145,2.0,1.0,100.0,1950000.0,Alexandria,Mandara,محمد عادل,No Link Was Provided,https://www.bayut.eg/en/property/details-50264...
3146,3.0,2.0,150.0,3950000.0,Alexandria,Laurent,Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...,https://www.bayut.eg/en/property/details-50259...
3147,3.0,2.0,150.0,3950000.0,Alexandria,Laurent,Nader Saleh,https://www.bayut.eg/en/brokers/nader-saleh-00...,https://www.bayut.eg/en/property/details-50259...
3148,5.0,5.0,400.0,10000000.0,Alexandria,King Mariout,عاشور,No Link Was Provided,https://www.bayut.eg/en/property/details-50258...


## Finally it Ends🥳🥳

In [ ]:
LastDayEd.to_csv("BayutData(Completed).csv", index=False, encoding='utf-8-sig')